<a href="https://colab.research.google.com/github/No-Qubit-Left-Behind/Control-Engineering-in-TF/blob/master/TF_Propagator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Propagator

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from google.colab import files
%tensorflow_version 2.x
#!pip install tensorflow==2.0.0-beta0
import tensorflow as tf
import numpy as np
import time
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0


In [9]:
class Propagator:
  def __init__(self, no_of_steps):
    x = -0.0602715189469 * tf.constant(
        [[0 + 0j, 0 + 1j], [0 + 1j, 0 + 0j]], dtype=tf.complex128
    )
    y = -0.0602715189469 * tf.constant(
        [[0, 1], [-1, 0]], dtype=tf.complex128
    )
    self.Rabi_strength = tf.Variable(0., dtype=tf.complex128)
    self.generators =  tf.stack([x, y])
    self.controls = tf.convert_to_tensor(
        np.ones((no_of_steps, 2)),
        dtype=tf.complex128
    )

    self.steps_power = int(np.floor(np.log2(no_of_steps)))
    bool_array = []
    length = no_of_steps
    for i in range(self.steps_power):
      bool_array.append(bool(np.mod(length, 2)))
      length = np.floor(length / 2)
    self.bool_array = bool_array   

  @tf.function
  def exponents(self):
    exponents = tf.linalg.tensordot(
        self.controls,
        self.generators,
        1
    )
    return exponents
  
  @tf.function
  def exponentials(self):
    exponents = self.exponents()
    return tf.linalg.expm(exponents)
    
  @tf.function
  def __call__(self):
    step_exps = self.exponentials()
    for i in range(self.steps_power):
      if self.bool_array[i]:
        odd_exp = step_exps[-1, :, :]
        step_exps = tf.linalg.matmul(
          step_exps[1::2, :, :],
          step_exps[0:-1, :, :][0::2, :, :]
        )
        step_exps = tf.concat([
            step_exps[0:-1, :, :],
            [tf.linalg.matmul(odd_exp ,step_exps[-1, :, :])]
        ], 0)
      else:
        step_exps = tf.linalg.matmul(
          step_exps[1::2, :, :],
          step_exps[0::2, :, :]
        )
    return tf.squeeze(step_exps)

propagator = Propagator(1372707)

start = time.time()
for i in range(10):
  propagator.Rabi_strength.assign(i)
  propagator().numpy()
end = time.time()
print(end-start)

67.8755111694336
